In [1]:
# Chapter 2
# Coding a single neuron.

In [2]:
# Input is a array/vector that will either be
# an actual training data or output of neurons from prev layer.
inputs = [1, 2, 3]

In [3]:
# Each input will also have a weight associated with it,
# this weight will be tuned later while training.
weights = [0.2, 0.8, -0.5]

# Next is bias, each neuron has just one bias value.
# Therefore, for now, our single neuron will have just one biasValue
bias = 2

In [4]:
output = (inputs[0]*weights[0] +
          inputs[1]*weights[1] +
          inputs[2]*weights[2] + bias)

print("Output: ", output)

Output:  2.3


A layer of Neurons

Each neuron in a layer takes exactly the same input — the input given to the layer (which can be either the training data or the output from the previous layer), but contains its own set of weights and its own bias, producing its own unique output. The layer’s output is a set of each of these outputs — one per each neuron. 

![alt text](imagename.png "Title")

In [8]:
# A neural layer with 3 neurons

inputs = [1,2,3,2.5]

# for each neuron we will have a set of weights
# in our case, 3 arrays of weights.
# remember that each weight value is associated with each input value
weights1 = [0.2, 0.8, -0.5, 1]
weights2 = [0.5, -0.91, 0.26, -0.5]
weights3 = [-0.26, -0.27, 0.17, 0.87]

# for each neuron we will have a single bias value
bias1 = 2
bias2 = 3
bias3 = 0.5

outputs = [
    # output for neuron 1
    inputs[0]*weights1[0] + 
    inputs[1]*weights1[1] + 
    inputs[2]*weights1[2] +
    inputs[3]*weights1[3] +
    bias1,

    # output for neuron 2
    inputs[0]*weights2[0] +
    inputs[1]*weights2[1] +
    inputs[2]*weights2[2] +
    inputs[3]*weights2[3] +
    bias2,

    # output for neuron 3
    inputs[0]*weights3[0] +
    inputs[1]*weights3[1] +
    inputs[2]*weights3[2] +
    inputs[3]*weights3[3] +
    bias3
]

print("Output: ", outputs)

Output:  [4.8, 1.21, 2.385]


In the above code, we have three sets of weights and three biases, which define three neurons. Each neuron is “connected” to the same inputs. The difference is in the separate weights and bias that each neuron applies to the input. This is called a fully connected neural network — every neuron in the current layer has connections to every neuron from the previous layer. This is a very common type of neural network, but it should be noted that there is no requirement to fully connect everything like this. At this point, we have only shown code for a single layer with very few neurons. Imagine coding many more layers and more neurons. This would get very challenging to code using our current methods. Instead, we could use a loop to scale and handle dynamically-sized inputs and layers. We’ve turned the separate weight variables into a list of weights so we can iterate over them, and we changed the code to use loops instead of the hardcoded operations.

In [13]:
inputs = [1, 2, 3, 2.5]
weights = [
    [0.2, 0.8, -0.5, 1],            #weights for #1 neuron
    [0.5, -0.91, 0.26, -0.5],       #weights for #2 neuron
    [-0.26, -0.27, 0.17, 0.87]      #weights for #3 neuron
]
bias = [2, 3, 0.5]

# output for current layer
outputs = []
for weightsOfNeuron, biasOfNeuron in zip(weights, bias):
    output = 0
    for weightValue, inputValue in zip(weightsOfNeuron, inputs):
        output += inputValue*weightValue
    outputs.append(output + biasOfNeuron)
    

print(outputs)

[4.8, 1.21, 2.385]


In [ ]:
# to be continued from page 34...